In [92]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
from scipy.stats import zscore
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import lightgbm as lgb
from bayes_opt import BayesianOptimization
import warnings, gc
# Any results you write to the current directory are saved as output.

['2016-soi-tax-stats', 'midterm']


In [93]:
def multi_merge(left,right,*args):
    start = args[0]
    for i in range(1,len(args)):
        start = start.merge(args[i], how = 'left', left_on = left, right_on = right)
    return start

def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(
        target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type not in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    # Regression
    return df[result].values.astype(np.float32), df[[target]].values.astype(np.float32)

> **Data Clean Part**

In [94]:
df_train = pd.read_csv('../input/midterm/train.csv')
df_test = pd.read_csv('../input/midterm/test.csv')
df_detail = pd.read_csv('../input/2016-soi-tax-stats/16zpallagi.csv')
df_detail = df_detail.loc[(df_detail['zipcode'] != 0) & (df_detail['zipcode'] != 99999)].reset_index(drop=True)

In [95]:
import re
column_list = list(df_detail.columns)
A_list = [column for column in column_list if re.match(r'^[A][0-9]{2,10}',column)]
N_list = [column for column in column_list if re.match(r'^[N][0-9]{2,10}',column)]
O_list = [column for column in column_list if (column not in A_list and column not in N_list)]

In [96]:
special = []
for name in A_list:
    if 'N' + name[1:] not in N_list:
        special.append(name)
df_zipcode = df_detail.loc[:,'zipcode']
df_detail_pair = df_detail.iloc[:,18:].drop(special+['SCHF'],axis=1)
df_detail_pair = pd.concat([df_zipcode,df_detail_pair],axis = 1)
for name in N_list:
    A_name = 'A' + name[1:]
    df_detail_pair[A_name] = df_detail_pair[A_name]*df_detail_pair[name]

In [97]:
df_detail_pair = df_detail_pair.groupby('zipcode').sum()
for name in N_list:
    A_name = 'A' + name[1:]
    df_detail_pair[A_name] = df_detail_pair[A_name]/df_detail_pair[name]
df_feature_1 = df_detail_pair.loc[:,['A' + name[1:] for name in N_list]].fillna(0)

In [98]:
df_special_feature_1 = df_detail.loc[:,special+['zipcode','N1']]
df_special_feature_1 = df_special_feature_1.groupby(by='zipcode').apply(lambda x: pd.Series({'avg_agi':sum(x['N1']*x[special[0]])/sum(x['N1']),'avg_item_r':sum(x['N1']*x[special[1]])/sum(x['N1'])}))

In [99]:
df_special_feature_2 = df_detail.iloc[:,:18]
for column in list(df_special_feature_2.columns)[5:]:
    df_special_feature_2[column] = df_special_feature_2[column]/df_special_feature_2['N1']
df_special_feature_2 = df_special_feature_2.fillna(0).drop(['STATEFIPS','STATE','agi_stub'],axis=1).groupby(by='zipcode').agg('mean')

In [100]:
df_full_detail = multi_merge('zipcode','zipcode',df_special_feature_2,df_special_feature_1,df_feature_1)
df_full_detail.head()

,N1,mars1,MARS2,MARS4,PREP,N2,NUMDEP,TOTAL_VITA,VITA,TCE,VITA_EIC,RAL,RAC,ELDERLY,avg_agi,avg_item_r,A02650,A00200,A00300,A00600,A00650,A00700,A00900,A01000,A01400,A01700,A02300,A02500,A26270,A02900,A03220,A03300,A03270,A03150,A03210,A03230,A03240,A04470,A17000,A18425,A18450,A18500,A18800,A18300,A19300,A19500,A19530,A19550,A19570,A19700,A20800,A04800,A05800,A09600,A05780,A07100,A07300,A07180,A07230,A07240,A07220,A07260,A09400,A85770,A85775,A09750,A10600,A59660,A59720,A11070,A10960,A11560,A06500,A10300,A85530,A85300,A11901,A11902
zipcode,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001,1496.666667,0.434868,0.479333,0.076106,0.606140,1.977950,0.541338,0.012041,0.002372,0.010261,0.000000,0.002990,0.073244,0.299588,80337.743875,37501.525612,81330.952116,64046.144595,515.501639,935.430108,692.549133,294.103960,2734.028302,1053.503876,2653.745763,8489.688995,663.387755,3433.469799,1939.941176,1094.099099,20.206897,430.000000,202.933333,173.400000,252.967742,91.600000,0.0,12004.932476,1093.792453,3271.446735,34.750000,2292.811847,162.421739,5676.422581,3443.579592,135.000000,20.615385,155.342105,29.00,1205.532520,775.687500,59757.267568,9143.719132,296.000000,42.352941,589.733333,6.076923,54.666667,158.733333,30.488372,361.652893,40.086957,453.287356,198.120000,197.875000,37.611111,10884.271879,1015.785714,872.860759,283.943396,102.039216,30.818182,8849.053237,9130.414365,61.0,89.0,747.731544,3429.103933
1002,1581.666667,0.426821,0.483702,0.075928,0.543996,1.945985,0.512028,0.021110,0.013754,0.007356,0.004028,0.002014,0.057039,0.397310,110209.094837,76528.651212,112762.510011,68397.574440,1365.477273,7195.936170,5828.594855,491.852041,9359.559633,7644.710884,5950.027027,17160.873832,357.482759,5472.768786,14372.985075,2988.672840,22.864865,2014.000000,709.490196,345.607143,230.739130,171.045455,478.0,26671.457584,1876.096386,8241.715493,45.900000,7165.760446,200.776978,15064.363402,4553.858268,203.333333,5.200000,50.571429,226.75,3884.156156,1680.297030,90529.577519,17208.319790,2920.580645,50.500000,795.161473,250.188406,55.558140,183.547368,36.613636,233.418605,279.500000,1371.350000,343.864865,313.536585,28.466667,18077.309922,1077.425743,951.927711,262.408163,143.475410,75.666667,17565.600846,18141.400534,478.0,913.0,2165.102679,3328.300000
1003,34.000000,0.400000,0.000000,0.000000,0.026667,0.493333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1144.470588,0.000000,1178.882353,1140.823529,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,39.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,420.285714,58.571429,0.000000,0.000000,11.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,124.375000,0.000000,0.000000,0.000000,24.000000,0.000000,50.571429,52.857143,0.0,0.0,0.000000,127.000000
1005,395.000000,0.349221,0.563934,0.072222,0.588392,2.197465,0.666593,0.000000,0.000000,0.000000,0.000000,0.000000,0.095656,0.234669,22167.666667,10815.708861,22489.105485,18147.203883,101.540230,255.414634,190.341463,79.781818,1259.212121,431.709677,595.120000,2172.040816,249.210526,775.472222,866.888889,390.523077,11.333333,0.000000,184.000000,217.000000,64.428571,42.000000,0.0,3605.024390,196.333333,944.545455,22.000000,705.000000,48.034483,1668.731707,1168.338028,0.000000,0.000000,49.545455,0.00,262.964912,287.000000,16397.639594,2451.994924,92.000000,30.000000,215.521127,2.333333,28.000000,52.764706,11.727273,131.500000,48.000000,177.384615,64.857143,78.666667,32.000000,2912.877193,232.360000,205.100000,81.692308,33.642857,25.000000,2333.698925,2440.118557,0.0,0.0,232.511111,942.876344
1007,1263.333333,0.371875,0.535202,0.074259,0.549149,2.118798,0.646966,0.010649,0.001536,0.009113,0.000000,0.002304,0.067025,0.274557,89435.886544,60582.518470,90681.472296,751

In [101]:
for name in df_full_detail.columns:
    if df_full_detail[name].dtype in ('float64','int64'):
        df_full_detail[name] = zscore(df_full_detail[name])
df_train = multi_merge('zipcode','zipcode',df_train,df_full_detail).drop(['id','zipcode'],axis=1)
df_test = multi_merge('zipcode','zipcode',df_test,df_full_detail).drop(['id','zipcode'],axis=1)
x,y = to_xy(df_train,'score')

**Tree Model**

In [ ]:
import lightgbm as lgb
from bayes_opt import BayesianOptimization
import warnings, gc
features = [c for c in df_train.columns if c not in ['score']]
df_target = df_train['score']
def lgb_cv(max_depth,
          num_leaves,
          min_data_in_leaf,
          feature_fraction,
          bagging_fraction,
          lambda_l1):
    folds = KFold(n_splits=5, shuffle=True, random_state=15)
    oof = np.zeros(df_train.shape[0])
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train.values, df_target.values)):
        print("fold n°{}".format(fold_))
        trn_data = lgb.Dataset(df_train.iloc[trn_idx][features],
                               label=df_target.iloc[trn_idx]
                              )
        val_data = lgb.Dataset(df_train.iloc[val_idx][features],
                               label=df_target.iloc[val_idx]
                              )
        param = {
            'num_leaves': int(num_leaves),
            'min_data_in_leaf': int(min_data_in_leaf), 
            'objective':'regression',
            'max_depth': int(max_depth),
            'learning_rate': 0.005,
            "boosting": "gbdt",
            "feature_fraction": feature_fraction,
            "bagging_freq": 1,
            "bagging_fraction": bagging_fraction ,
            "bagging_seed": 11,
            "metric": 'rmse',
            "lambda_l1": lambda_l1,
            "verbosity": -1
        }
        clf = lgb.train(param,
                        trn_data,
                        10000,
                        valid_sets = [trn_data, val_data],
                        verbose_eval=200,
                        early_stopping_rounds = 100)
        oof[val_idx] = clf.predict(df_train.iloc[val_idx][features],
                                   num_iteration=clf.best_iteration)
        del clf, trn_idx, val_idx
        gc.collect()
    return -metrics.mean_squared_error(oof, df_target)**0.5

In [ ]:
LGB_BO = BayesianOptimization(lgb_cv, {
    'max_depth': (4, 10),
    'num_leaves': (5, 130),
    'min_data_in_leaf': (10, 150),
    'feature_fraction': (0.7, 1.0),
    'bagging_fraction': (0.7, 1.0),
    'lambda_l1': (0, 6)
    })

In [ ]:
print('-'*126)
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    LGB_BO.maximize(init_points=2, n_iter=4, acq='ei', xi=0.0)

In [ ]:
features = [c for c in df_train.columns if c not in ['score']]
df_target = df_train['score']
param = {
            'num_leaves': int(52),
            'min_data_in_leaf': int(12), 
            'objective':'regression',
            'max_depth': int(6),
            'learning_rate': 0.005,
            "boosting": "gbdt",
            "feature_fraction": 0.7066,
            "bagging_freq": 1,
            "bagging_fraction": 0.8086,
            "bagging_seed": 11,
            "metric": 'rmse',
            "lambda_l1": 3.651,
            "random_state": 11,
            "verbosity": -1
        }
folds = KFold(n_splits=5, shuffle=True, random_state=15)
oof = np.zeros(len(df_train))
predictions = np.zeros(len(df_test))
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train.values, df_target.values)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(df_train.iloc[trn_idx][features],
                           label=df_target.iloc[trn_idx]
                          )
    val_data = lgb.Dataset(df_train.iloc[val_idx][features],
                           label=df_target.iloc[val_idx]
                          )

    num_round = 10000
    clf = lgb.train(param,
                    trn_data,
                    num_round,
                    valid_sets = [trn_data, val_data],
                    verbose_eval=100,
                    early_stopping_rounds = 200)
    
    oof[val_idx] = clf.predict(df_train.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(df_test[features], num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.5f}".format(metrics.mean_squared_error(oof, df_target)**0.5))

In [ ]:
submit_csv = pd.read_csv('../input/midterm/test.csv')
submit_csv = submit_csv.drop('zipcode',axis=1)
submit_csv['score'] = predictions
submit_csv.to_csv('csv_to_submit_80.csv', index = False)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
cols = (feature_importance_df[["feature", "importance"]]
        .groupby("feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:1000].index)

best_features = feature_importance_df.loc[feature_importance_df.feature.isin(cols)]

plt.figure(figsize=(14,25))
sns.barplot(x="importance",
            y="feature",
            data=best_features.sort_values(by="importance",
                                           ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.savefig('lgbm_importances.png')

**Model Part (Neural)**

In [107]:
#features = list(cols)[:10]
#features_train = features + ['score']
#df_train = df_train.loc[:,features_train]
#df_test = df_test.loc[:,features]
#x,y = to_xy(df_train,'score')

kf = KFold(5)
oos_y = []
oos_pred = []
fold = 0
for train,test in kf.split(x):
    fold += 1
    print('Fold #{}'.format(fold))
    x_train = x[train]
    y_train = y[train]
    x_test = x[test]
    y_test = y[test]
    
    model = Sequential()
    model.add(Dense(100, input_dim=x.shape[1], activation='relu'))
    model.add(Dense(75, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-2, patience=10, verbose=1, mode='auto')
    checkpointer = ModelCheckpoint(filepath="midterm_model1_best", verbose=0, save_best_only=True)
    model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor,checkpointer],verbose=0,epochs=250)
    
    pred = model.predict(x_test)
    oos_y.append(y_test)
    oos_pred.append(pred)
    score = np.sqrt(metrics.mean_squared_error(pred,y_test))
    print("Fold score (RMSE): {}".format(score))

oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
score = metrics.mean_squared_error(oos_pred,oos_y)
print("Final score (RMSE): {}".format(np.sqrt(score)))

df_test = pd.read_csv('../input/midterm/test.csv')
df_test = multi_merge('zipcode','zipcode',df_test,df_full_detail).drop(['id','zipcode'],axis=1)
true_test = df_test.values.astype(np.float32)
pred_test = model.predict(true_test)
pred_test = model.predict(true_test)
final_test_score = np.concatenate(pred_test)

df_test['score'] = final_test_score
df_test = df_test.loc[:,['id','score']]
df_test.to_csv('csv_to_submit_80_neural.csv', index = False)


Fold #1
Epoch 00029: early stopping
Fold score (RMSE): 5.9513936042785645
Fold #2
Epoch 00025: early stopping
Fold score (RMSE): 6.886742115020752
Fold #3
Epoch 00027: early stopping
Fold score (RMSE): 7.139791488647461
Fold #4
Epoch 00029: early stopping
Fold score (RMSE): 6.25360107421875
Fold #5
Epoch 00017: early stopping
Fold score (RMSE): 7.392597675323486
Final score (RMSE): 6.746560096740723


/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)
